# DICE 2013R Implementation

This is mostly a transcription of the `DICE2013R_100413_vanilla.gms` file.

Differences in the output do exist, since we use the open source **Ipopt** NLP solver instead of the commercial **CONOPT** solver.
All values and functions that can be compared have been checked and validated to match 1:1 to the GAMS implementation.
Interestingly enough, the commercial solver under-performs compared to the open source one, yielding a maximum utility of 2689.1762 (CONOPT) compared to 2690.2447128731565 (IPOPT) for the optimal control run (`opt = true`).

In [ ]:
# For Solving #
using JuMP;
#Ipopt is distributed under the EPL.
#We don't package it here though, and assume you have this set up on your system already.
using Ipopt; 

# For Plotting #
using Plots;
plotlyjs(); #Feel free to use a different backend

# For Comparing with GAMS data
using DataFrames;
using CSV;

## Scalars

In [ ]:
# Time Step #
N = 60; #Number of years to calculate (from 2010 onwards)
tstep = 5; #Years per Period

# If optimal control #
opt = true; #If optimized set true if base set false

# Preferences #
α = 1.45; #Elasticity of marginal utility of consumption
ρ = 0.015; #Initial rate of social time preference per year ρ

# Population and technology #
γₑ = 0.3; #Capital elasticity in production function
pop₀ = 6838; #Initial world population (millions)
popadj = 0.134; #Growth rate to calibrate to 2050 pop projection
popasym = 10500; #Asymptotic population (millions)
δk = 0.1; #Depreciation rate on capital (per year)
q₀ = 63.69; #Initial world gross output (trill 2005 USD)
k₀ = 135.0; #Initial capital value (trill 2005 USD)
a₀ = 3.80; #Initial level of total factor productivity
ga₀ = 0.079; #Initial growth rate for TFP per 5 years
δₐ = 0.006; #Decline rate of TFP per 5 years
#Optimal long-run savings rate used for transversality
optlrsav = (δk + .004)/(δk + .004α + ρ)*γₑ;

# Emissions parameters #
gσ₁ = -0.01; #Initial growth of sigma (continuous per year)
δσ = -0.001; #Decline rate of decarbonization per period
eland₀ = 3.3; #Carbon emissions from land 2010 (GtCO2 per year)
deland = 0.2; #Decline rate of land emissions (per period)
e₀ = 33.61; #Industrial emissions 2010 (GtCO2 per year)
miu₀ = 0.039; #Initial emissions control rate for base case 2010

# Carbon cycle #
# Initial Conditions
mat₀ = 830.4; #Initial Concentration in atmosphere 2010 (GtC)
mu₀ = 1527.0; #Initial Concentration in upper strata 2010 (GtC)
ml₀ = 10010.0; #Initial Concentration in lower strata 2010 (GtC)
mateq = 588.0; #Equilibrium concentration atmosphere  (GtC)
mueq = 1350.0; #Equilibrium concentration in upper strata (GtC)
mleq = 10000.0; #Equilibrium concentration in lower strata (GtC)

# Flow paramaters
#Carbon cycle transition matrix
ϕ₁₂ = .088;
ϕ₂₃ = 0.0025; #Carbon cycle transition matrix
# Parameters for long-run consistency of carbon cycle
ϕ₁₁ = 1 - ϕ₁₂;
ϕ₂₁ = ϕ₁₂*mateq/mueq;
ϕ₂₂ = 1 - ϕ₂₁ - ϕ₂₃;
ϕ₃₂ = ϕ₂₃*mueq/mleq;
ϕ₃₃ = 1 - ϕ₃₂;
σ₀ = e₀/(q₀*(1-miu₀)); #Carbon intensity 2010 (kgCO2 per output 2005 USD 2010)

# Climate model parameters #
t2xco2 = 2.9; #Equilibrium temp impact (oC per doubling CO2)
fₑₓ0 = 0.25; #2010 forcings of non-CO2 GHG (Wm-2)
fₑₓ1 = 0.7; #2100 forcings of non-CO2 GHG (Wm-2)
tocean₀ = 0.0068; #Initial lower stratum temp change (C from 1900)
tatm₀ = 0.8; #Initial atmospheric temp change (C from 1900)
ξ₁ = 0.098; #Climate equation coefficient for upper level
ξ₃ = 0.088; #Transfer coefficient upper to lower stratum
ξ₄ = 0.025; #Transfer coefficient for lower level
η = 3.8; #Forcings of equilibrium CO2 doubling (Wm-2)

λ = η/t2xco2; #Climate model parameter

# Climate damage parameters #
ψ₁ = 0.0; #Damage intercept
ψ₂ = 0.00267; #Damage quadratic term
ψ₃ = 2.0; #Damage exponent

# Abatement cost #
θ₂ = 2.8; #Exponent of control cost function
pback = 344.0; #Cost of backstop 2005$ per tCO2 2010
gback = 0.025; #Initial cost decline backstop cost per period
limμ = 1.2; #Upper limit on control rate after 2150
tnopol = 45.0; #Period before which no emissions controls base
cprice₀ = 1.0; #Initial base carbon price (2005$ per tCO2)
gcprice = 0.02; #Growth rate of base carbon price per year

# Participation parameters #
periodfullpart = 21; #Period at which have full participation
partfract2010 = 1; #Fraction of emissions under control in 2010
partfractfull = 1; #Fraction of emissions under control at full time

# Availability of fossil fuels #
fosslim  = 6000.0; #Maximum cumulative extraction fossil fuels (GtC)

# Scaling and inessential parameters #
# Note that these are unnecessary for the calculations but are for convenience
scale1 = 0.016408662; #Multiplicative scaling coefficient
scale2 = -3855.106895; #Additive scaling coefficient

## Vectors

In [ ]:
# Backstop price
pbacktime = Array{Float64}(N);
# Growth rate of productivity from
gₐ = Array{Float64}(N);
# Emissions from deforestation
Etree = Array{Float64}(N);
# Average utility social discount rate
rr = Array{Float64}(N);
# Carbon price in base case
cpricebase = Array{Float64}(N);

for i in 1:N
    pbacktime[i] = pback*(1-gback)^(i-1);
    gₐ[i] = ga₀*exp.(-δₐ*5*(i-1));
    Etree[i] = eland₀*(1-deland)^(i-1);
    rr[i] = 1/((1+ρ)^(tstep*(i-1)));
    cpricebase[i] = cprice₀*(1+gcprice)^(5*(i-1));
end

# Initial conditions and offset required
# Level of population and labor
L = Array{Float64}(N);
L[1] = pop₀;
# Level of total factor productivity
A = Array{Float64}(N);
A[1] = a₀;
# Change in sigma (cumulative improvement of energy efficiency)
gσ = Array{Float64}(N);
gσ[1] = gσ₁;
# CO2-equivalent-emissions output ratio
σ = Array{Float64}(N);
σ[1] = σ₀;


for i in 1:N-1
    L[i+1] = L[i]*(popasym/L[i])^popadj;
    A[i+1] = A[i]/(1-gₐ[i]);
    gσ[i+1] = gσ[i]*((1+δσ)^tstep);
    σ[i+1] = σ[i]*exp.(gσ[i]*tstep);
end

# Adjusted cost for backstop
θ₁ = Array{Float64}(N); 
# Exogenous forcing for other greenhouse gases
fₑₓ = Array{Float64}(N);
# Fraction of emissions in control regime
partfract = Array{Float64}(N);

for i in 1:N
    θ₁[i] = pbacktime[i]*σ[i]/θ₂/1000.0;
    fₑₓ[i] = if i < 19
                     fₑₓ0+(1/18)*(fₑₓ1-fₑₓ0)*(i-1)
                 else
                     fₑₓ1-fₑₓ0
                 end;
    partfract[i] = if i <= periodfullpart
                        partfract2010+(partfractfull-partfract2010)*(i-1)/periodfullpart
                   else
                        partfractfull
                   end;
end
partfract[1] = partfract2010;

## Model

Word of warning: if you `print(CO₂)` in this notebook you're gonna have a bad time.
It's apparently too big to print or something.
The thread just locks up/spools forever.

There's a *limit output* notebook extension which can truncate things for you.
It seems to be necessary if you want to use notebooks which have actual output.

In [ ]:
CO₂ = Model(solver = IpoptSolver(print_level=3, max_iter=99900,print_frequency_iter=50));
# Rate limit
μ_ubound(t) = if t < 30
                    1.0
                else
                    limμ*partfract[t]
                end;

# Variables #
@variable(CO₂, 0.0 <= μ[i=1:N] <= μ_ubound(i)); # Emission control rate GHGs
@variable(CO₂, FORC[1:N]); # Increase in radiative forcing (watts per m2 from 1900)
@variable(CO₂, 0.0 <= Tₐₜ[1:N] <= 40.0); # Increase temperature of atmosphere (degrees C from 1900)
@variable(CO₂, -1.0 <= Tₗₒ[1:N] <= 20.0); # Increase temperatureof lower oceans (degrees C from 1900)
@variable(CO₂, Mₐₜ[1:N] >= 10.0); # Carbon concentration increase in atmosphere (GtC from 1750)
@variable(CO₂, Mᵤₚ[1:N] >= 100.0); # Carbon concentration increase in shallow oceans (GtC from 1750)
@variable(CO₂, Mₗₒ[1:N] >= 1000.0); # Carbon concentration increase in lower oceans (GtC from 1750)
@variable(CO₂, E[1:N]); # Total CO2 emissions (GtCO2 per year)
@variable(CO₂, Eind[1:N]); # Industrial emissions (GtCO2 per year)
@variable(CO₂, C[1:N] >= 2.0); # Consumption (trillions 2005 US dollars per year)
@variable(CO₂, K[1:N] >= 1.0); # Capital stock (trillions 2005 US dollars)
@variable(CO₂, CPC[1:N] >= 0.01); #  Per capita consumption (thousands 2005 USD per year)
@variable(CO₂, I[1:N] >= 0.0); # Investment (trillions 2005 USD per year)
@variable(CO₂, S[1:N]); # Gross savings rate as fraction of gross world product
@variable(CO₂, RI[1:N]); # Real interest rate (per annum)
@variable(CO₂, Y[1:N] >= 0.0); # Gross world product net of abatement and damages (trillions 2005 USD per year)
@variable(CO₂, YGROSS[1:N] >= 0.0); # Gross world product GROSS of abatement and damages (trillions 2005 USD per year)
@variable(CO₂, YNET[1:N]); # Output net of damages equation (trillions 2005 USD per year)
@variable(CO₂, DAMAGES[1:N]); # Damages (trillions 2005 USD per year)
@variable(CO₂, Ω[1:N]); # Damages as fraction of gross output
@variable(CO₂, Λ[1:N]); # Cost of emissions reductions  (trillions 2005 USD per year)
@variable(CO₂, MCABATE[1:N]); # Marginal cost of abatement (2005$ per ton CO2)
@variable(CO₂, CCA[1:N] <= fosslim); # Cumulative industrial carbon emissions (GTC)
@variable(CO₂, PERIODU[1:N]); # One period utility function
@variable(CO₂, CPRICE[i=1:N]); # Carbon price (2005$ per ton of CO2)
@variable(CO₂, CEMUTOTPER[1:N]); # Period utility
@variable(CO₂, UTILITY); # Welfare function

# Base carbon price if base, otherwise optimized
# Warning: If parameters are changed, the next equation might make base case infeasible.
# If so, reduce tnopol so that don't run out of resources.
setupperbound(CPRICE[1], cpricebase[1]);  
for i in 2:N
    if !opt
        setupperbound(CPRICE[i], cpricebase[i]);  
    elseif i > tnopol
        setupperbound(CPRICE[i], 1000.0);  
    end
end

# Equations #
# Emissions Equation
eeq = @constraint(CO₂, [i=1:N], E[i] == Eind[i] + Etree[i]);
# Industrial Emissions
@constraint(CO₂, [i=1:N], Eind[i] == σ[i] * YGROSS[i] * (1-μ[i]));
# Radiative forcing equation
@NLconstraint(CO₂, [i=1:N], FORC[i] == η * (log(Mₐₜ[i]/588.0)/log(2)) + fₑₓ[i]);
# Equation for damage fraction
@NLconstraint(CO₂, [i=1:N], Ω[i] == ψ₁*Tₐₜ[i]+ψ₂*Tₐₜ[i]^ψ₃);
# Damage equation
@constraint(CO₂, [i=1:N], DAMAGES[i] == YGROSS[i]*Ω[i]);
# Cost of exissions reductions equation
@NLconstraint(CO₂, [i=1:N], Λ[i] == YGROSS[i] * θ₁[i] * μ[i]^θ₂ * partfract[i]^(1-θ₂));
# Equation for MC abatement
@NLconstraint(CO₂, [i=1:N], MCABATE[i] == pbacktime[i] * μ[i]^(θ₂-1));
# Carbon price equation from abatement
@NLconstraint(CO₂, [i=1:N], CPRICE[i] == pbacktime[i] * (μ[i]/partfract[i])^(θ₂-1));
# Output gross equation
@NLconstraint(CO₂, [i=1:N], YGROSS[i] == A[i]*(L[i]/1000.0)^(1-γₑ)*K[i]^γₑ);
# Output net of damages equation
@constraint(CO₂, [i=1:N], YNET[i] == YGROSS[i]*(1-Ω[i]));
# Output net equation
@constraint(CO₂, [i=1:N], Y[i] == YNET[i] - Λ[i]);
# Consumption equation
cc = @constraint(CO₂, [i=1:N], C[i] == Y[i] - I[i]);
# Per capita consumption definition
@constraint(CO₂, [i=1:N], CPC[i] == 1000.0 * C[i] / L[i]);
# Savings rate equation
@constraint(CO₂, [i=1:N], I[i] == S[i] * Y[i]);
# Period utility
@constraint(CO₂, [i=1:N], CEMUTOTPER[i] == PERIODU[i] * L[i] * rr[i]);
# Instantaneous utility function equation
@NLconstraint(CO₂, [i=1:N], PERIODU[i] == ((C[i]*1000.0/L[i])^(1-α)-1)/(1-α)-1);

# Equations (offset) #
# Cumulative carbon emissions
@constraint(CO₂, [i=1:N-1], CCA[i+1] == CCA[i] + Eind[i]*5/3.666);
# Atmospheric concentration equation
@constraint(CO₂, [i=1:N-1], Mₐₜ[i+1] == Mₐₜ[i]*ϕ₁₁ + Mᵤₚ[i]*ϕ₂₁ + E[i]*(5/3.666));
# Lower ocean concentration
@constraint(CO₂, [i=1:N-1], Mₗₒ[i+1] == Mₗₒ[i]*ϕ₃₃ + Mᵤₚ[i]*ϕ₂₃);
# Shallow ocean concentration
@constraint(CO₂, [i=1:N-1], Mᵤₚ[i+1] == Mₐₜ[i]*ϕ₁₂ + Mᵤₚ[i]*ϕ₂₂ + Mₗₒ[i]*ϕ₃₂);
# Temperature-climate equation for atmosphere
@constraint(CO₂, [i=1:N-1], Tₐₜ[i+1] == Tₐₜ[i] + ξ₁*((FORC[i+1]-λ*Tₐₜ[i])-(ξ₃*(Tₐₜ[i]-Tₗₒ[i]))));
# Temperature-climate equation for lower oceans
@constraint(CO₂, [i=1:N-1], Tₗₒ[i+1] == Tₗₒ[i] + ξ₄*(Tₐₜ[i]-Tₗₒ[i]));
# Capital balance equation
@constraint(CO₂, [i=1:N-1], K[i+1] <= (1-δk)^tstep * K[i] + tstep*I[i]);
# Interest rate equation
@NLconstraint(CO₂, [i=1:N-1], RI[i] == (1+ρ)*(CPC[i+1]/CPC[i])^(α/tstep)-1);

# Savings rate for asympotic equilibrium
@constraint(CO₂, S[i=51:N] .== optlrsav);
# Initial conditions
@constraint(CO₂, CCA[1] == 90.0);
@constraint(CO₂, K[1] == k₀);
@constraint(CO₂, Mₐₜ[1] == mat₀);
@constraint(CO₂, Mᵤₚ[1] == mu₀);
@constraint(CO₂, Mₗₒ[1] == ml₀);
@constraint(CO₂, Tₐₜ[1] == tatm₀);
@constraint(CO₂, Tₗₒ[1] == tocean₀);

@constraint(CO₂, UTILITY == tstep * scale1 * sum(CEMUTOTPER[i] for i=1:N) + scale2);

# Objective function
@objective(CO₂, Max, UTILITY);

In [ ]:
status = solve(CO₂)
status2 = solve(CO₂)
status3 = solve(CO₂)

In [ ]:
# Model outputs #
tatm = getvalue(Tₐₜ); # Atmospheric Temperature (deg C above preindustrial)
forc = getvalue(FORC); # Total Increase in Forcing (Watts per Meter2, preindustrial)
tocean = getvalue(Tₗₒ);# Lower Ocean Temperature (deg C above preindustrial)
# Calculate social cost of carbon
scc = -1000.0.*getdual(eeq)./getdual(cc);

In [ ]:
# Plot major outputs #
years = 2005+(tstep*(1:N));
a = plot(years,scc, ylabel="\$ (trillion)", title="SCC")
b = plot(years,tatm, ylabel="°C/T0", title="Atmos Temp")
c = plot(years,forc, xlabel="Years", ylabel="Wm^{-2}/T_0", title="Forcing")
d = plot(years,tocean, xlabel="Years", ylabel="°C/T0", title="Low Ocean Temp")
plot(a,b,c,d,layout=(2,2),legend=false)

## Comparision

To compare our values to the GAMS version, I've run the optimal version and output some of the more important variables. Feel free to check this yourself if you have a GAMS license and CONOPT.

In [ ]:
#Load in the results of an 'optimal=true' gams run.
gams = CSV.read("Vanilla2013RResults.csv", transpose=true)
names!(gams, [:Period,:Year,:SCC,:TATM,:FORCE,:TOCEAN]);

In [ ]:
a = plot(years,scc, ylabel="\$ (trillion)", title="SCC")
plot!(years, gams[:SCC])
b = plot(years,tatm, ylabel="°C/T₀", title="Atmos Temp")
plot!(gams[:Year],gams[:TATM])
c = plot(years,forc, xlabel="Years", ylabel="Wm⁻²/T₀", title="Forcing")
plot!(gams[:Year],gams[:FORCE])
d = plot(years,tocean, xlabel="Years", ylabel="°C/T₀", title="Low Ocean Temp")
plot!(gams[:Year],gams[:TOCEAN])
plot(a,b,c,d,layout=(2,2),legend=false)

## Troubleshooting

This is output cleaned up from the GAMS `lst` output file of an optimised run. 

In [ ]:
vars = CSV.read("Vanilla2013RVars.csv")

In [ ]:
a = plot(getvalue(μ), title="μ")
plot!(vars[:MIU])
b = plot(getvalue(FORC), title="FORC")
plot!(vars[:FORC])
c = plot(getvalue(Tₐₜ), title="Tₐₜ")
plot!(vars[:TATM])
d = plot(getvalue(Tₗₒ), title="Tₗₒ")
plot!(vars[:TOCEAN])
e = plot(getvalue(Mₐₜ), title="Mₐₜ")
plot!(vars[:MAT])
f = plot(getvalue(Mᵤₚ), title="Mᵤₚ")
plot!(vars[:MU])
g = plot(getvalue(Mₗₒ), title="Mₗₒ")
plot!(vars[:ML])
h = plot(getvalue(E), title="E")
plot!(vars[:E])
i = plot(getvalue(Eind), title="Eind")
plot!(vars[:EIND])
j = plot(getvalue(C), title="C")
plot!(vars[:C])
k = plot(getvalue(K), title="K")
plot!(vars[:K])
l = plot(getvalue(CPC), title="CPC")
plot!(vars[:CPC])
m = plot(getvalue(I), title="I")
plot!(vars[:I])
n = plot(getvalue(S), title="S")
plot!(vars[:S])
o = plot(getvalue(RI), title="RI")
plot!(vars[:RI])
p = plot(getvalue(Y), title="Y")
plot!(vars[:Y])
q = plot(getvalue(YGROSS), title="YGROSS")
plot!(vars[:YGROSS])
r = plot(getvalue(YNET), title="YNET")
plot!(vars[:YNET])
s = plot(getvalue(DAMAGES), title="DAMAGES")
plot!(vars[:DAMAGES])
t = plot(getvalue(Ω), title="Ω")
plot!(vars[:DAMFRAC])
u = plot(getvalue(Λ), title="Λ")
plot!(vars[:ABATECOST])
v = plot(getvalue(MCABATE), title="MCABATE")
plot!(vars[:MCABATE])
w = plot(getvalue(CCA), title="CCA")
plot!(vars[:CCA])
x = plot(getvalue(PERIODU), title="PERIODU")
plot!(vars[:PERIODU])
y = plot(getvalue(CPRICE), title="CPRICE")
plot!(vars[:CPRICE])
z = plot(getvalue(CEMUTOTPER), title="CEMUTOTPER")
plot!(vars[:CEMUTOTPER])
plot(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,layout=(13,2),legend=false,label=["Julia" "GAMS"],size=(700,2500))

## Conclusion of Troubleshooting Results

Since we're maximising for utility, we can safely say that the Ipopt solution is actually better than the Conopt. All differences in the GAMS/Julia solution arise from small multiplicative differences.

- **CONOPT** finds `UTILITY = 2689.1762`
- **Ipopt** finds `UTILITY = 2690.2447128731565`